## Getting Started with Prompt Management and Flows for Amazon Bedrock

This example shows you how to get started with Prompt Management and Prompt Flows in Amazon Bedrock.

[Amazon Bedrock Prompt Management](https://docs.aws.amazon.com/bedrock/latest/userguide/prompt-management.html) streamlines the creation, evaluation, deployment, and sharing of prompts in the Amazon Bedrock console and via APIs in the SDK. This feature helps developers and business users obtain the best responses from foundation models for their specific use cases.

[Amazon Bedrock Prompt Flows](https://docs.aws.amazon.com/bedrock/latest/userguide/flows.html) allows you to easily link multiple foundation models (FMs), prompts, and other AWS services, reducing development time and effort. It introduces a visual builder in the Amazon Bedrock console and a new set of APIs in the SDK, that simplifies the creation of complex generative AI workflows.

Let's start by making sure we have the lastest version of the Amazon Bedrock SDK, importing the libraries, and setting-up the client.

In [1]:
# Only run this the first time...
!pip3 install boto3 botocore -qU

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.33.24 requires botocore==1.34.142, but you have botocore 1.34.155 which is incompatible.


In [1]:
import boto3
from datetime import datetime
import json

Note the Prompt Management and Flows features are part of the Bedrock Agent SDK.

In [2]:
bedrock_agent = boto3.client(service_name = "bedrock-agent", region_name = "us-west-2")

## Prompt Management

### Create and Manage Prompts

Let's create a sample prompt, in this case for a simple translation task.

In [3]:
response = bedrock_agent.create_prompt(
    name = f"MyTestPrompt-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    description = "This is my test prompt for the customer service use case",
    variants = [
        {
            "inferenceConfiguration": {
                "text": {
                    "maxTokens": 3000,
                    "temperature": 0,
                    "topP": 0.999,
                    "topK": 250,
                }
            },
            "modelId": "anthropic.claude-3-haiku-20240307-v1:0",
            "name": "variant-001",
            "templateConfiguration": {
                "text": {
                    "inputVariables": [
                        {
                            "name": "input"
                        }

                    ],
                    "text": "You're a customer service agent for the ecommerce company Octank. Answer the following user query in a friendly and direct way: {{input}}"
                }
            },
            "templateType": "TEXT"
        }
    ],
    defaultVariant = "variant-001"
)
print(json.dumps(response, indent=2, default=str))
promptId = response["id"]
promptArn = response["arn"]
promptName = response["name"]
print(f"Prompt ID: {promptId}\nPrompt ARN: {promptArn}\nPrompt Name: {promptName}")

{
  "ResponseMetadata": {
    "RequestId": "ff30ca7b-456b-4d31-8aaa-de5803aa0187",
    "HTTPStatusCode": 201,
    "HTTPHeaders": {
      "date": "Wed, 07 Aug 2024 18:40:25 GMT",
      "content-type": "application/json",
      "content-length": "1110",
      "connection": "keep-alive",
      "x-amzn-requestid": "ff30ca7b-456b-4d31-8aaa-de5803aa0187",
      "x-amz-apigw-id": "cJrf7F5QPHcECoA=",
      "x-amzn-trace-id": "Root=1-66b3bf98-28d5a3cc10a7103b5eff4718"
    },
    "RetryAttempts": 0
  },
  "arn": "arn:aws:bedrock:us-west-2:376567470296:prompt/LEN4YMSXEQ",
  "createdAt": "2024-08-07 18:40:24.919964+00:00",
  "defaultVariant": "variant-001",
  "description": "This is my test prompt for the customer service use case",
  "id": "LEN4YMSXEQ",
  "name": "MyTestPrompt-20240807-184024",
  "updatedAt": "2024-08-07 18:40:24.919964+00:00",
  "variants": [
    {
      "inferenceConfiguration": {
        "text": {
          "maxTokens": 3000,
          "temperature": 0.0,
          "topK": 250

Now that we have a draft prompt, we can create versions from it.

In [4]:
response = bedrock_agent.create_prompt_version(
    promptIdentifier = promptId
)
print(json.dumps(response, indent=2, default=str))

{
  "ResponseMetadata": {
    "RequestId": "79ed7ff3-01f9-400d-ac41-345f0e483e56",
    "HTTPStatusCode": 201,
    "HTTPHeaders": {
      "date": "Wed, 07 Aug 2024 18:40:30 GMT",
      "content-type": "application/json",
      "content-length": "1035",
      "connection": "keep-alive",
      "x-amzn-requestid": "79ed7ff3-01f9-400d-ac41-345f0e483e56",
      "x-amz-apigw-id": "cJrgwErqvHcENxQ=",
      "x-amzn-trace-id": "Root=1-66b3bf9e-3f2219af0e9ceec67a24a9ea"
    },
    "RetryAttempts": 0
  },
  "arn": "arn:aws:bedrock:us-west-2:376567470296:prompt/LEN4YMSXEQ:1",
  "createdAt": "2024-08-07 18:40:30.249719+00:00",
  "defaultVariant": "variant-001",
  "id": "LEN4YMSXEQ",
  "name": "MyTestPrompt-20240807-184024",
  "updatedAt": "2024-08-07 18:40:30.249719+00:00",
  "variants": [
    {
      "inferenceConfiguration": {
        "text": {
          "maxTokens": 3000,
          "temperature": 0.0,
          "topK": 250,
          "topP": 0.9990000128746033
        }
      },
      "modelId": 

Here we can see the list of prompts in our Prompt Library or catalog.

In [5]:
response = bedrock_agent.list_prompts(
    maxResults = 10
)
print(json.dumps(response["promptSummaries"], indent=2, default=str))

[
  {
    "arn": "arn:aws:bedrock:us-west-2:376567470296:prompt/X301ZJ1MC8",
    "createdAt": "2024-07-28 17:00:18.656677+00:00",
    "description": "This is my prompt for customer service",
    "id": "X301ZJ1MC8",
    "name": "customer-service",
    "updatedAt": "2024-08-07 01:31:49.962047+00:00",
    "version": "DRAFT"
  },
  {
    "arn": "arn:aws:bedrock:us-west-2:376567470296:prompt/QAY0CQ2E9M",
    "createdAt": "2024-08-07 12:35:19.177682+00:00",
    "description": "Prompt to make a playlist given a genre and number of songs.",
    "id": "QAY0CQ2E9M",
    "name": "MakePlaylist",
    "updatedAt": "2024-08-07 12:50:48.864131+00:00",
    "version": "DRAFT"
  },
  {
    "arn": "arn:aws:bedrock:us-west-2:376567470296:prompt/609YX79AJ7",
    "createdAt": "2024-08-07 17:27:19.110749+00:00",
    "description": "This is my test prompt for the customer service use case",
    "id": "609YX79AJ7",
    "name": "MyTestPrompt-20240807-172718",
    "updatedAt": "2024-08-07 17:27:19.110749+00:00",


We can also read the details of any of our prompts.

In [6]:
response = bedrock_agent.get_prompt(
    promptIdentifier = promptId,
    promptVersion = "1"
)
print(json.dumps(response, indent=2, default=str))

{
  "ResponseMetadata": {
    "RequestId": "b82d44c6-fd4e-468b-98b2-bd5340ef0126",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 07 Aug 2024 18:40:39 GMT",
      "content-type": "application/json",
      "content-length": "1035",
      "connection": "keep-alive",
      "x-amzn-requestid": "b82d44c6-fd4e-468b-98b2-bd5340ef0126",
      "x-amz-apigw-id": "cJriJGuKvHcEe8w=",
      "x-amzn-trace-id": "Root=1-66b3bfa7-6eebcaa64583d8fa75cf456a"
    },
    "RetryAttempts": 0
  },
  "arn": "arn:aws:bedrock:us-west-2:376567470296:prompt/LEN4YMSXEQ:1",
  "createdAt": "2024-08-07 18:40:30.249719+00:00",
  "defaultVariant": "variant-001",
  "id": "LEN4YMSXEQ",
  "name": "MyTestPrompt-20240807-184024",
  "updatedAt": "2024-08-07 18:40:30.249719+00:00",
  "variants": [
    {
      "inferenceConfiguration": {
        "text": {
          "maxTokens": 3000,
          "temperature": 0.0,
          "topK": 250,
          "topP": 0.9990000128746033
        }
      },
      "modelId": 

-------

## Prompt Flows

Now that we've learned how to create and manage prompts, we can continue exploring how to build generative AI applications logic by creating workflows. For this, we'll rely on Prompt Flows for Amazon Bedrock.

### Create and Manage Flows

Let's create a simple flow that will load a prompt from our catalog. Note you can also create more complex flows involving chaining of steps, and conditions for dynamically routing, but let's keep it simple for now.

***Pre-requisite: For using Flows you need to make sure you have the proper AWS IAM permissions in place. You can check details in the [How Prompt Flows for Amazon Bedrock works](https://docs.aws.amazon.com/bedrock/latest/userguide/flows-how-it-works.html) documentation.***

In [8]:
### REPLACE WITH YOUR AWS IAM ROLE WITH FLOWS FOR BEDROCK PERMISSIONS
#flow_role = [REPLACE_WITH_YOUR_ROLE_ARN]
flow_role = 'arn:aws:iam::376567470296:role/service-role/AmazonBedrockExecutionRoleForFlows_X8IXBUVR83R'

In [9]:
response = bedrock_agent.create_flow(
    name = f"MyTestFlow-{datetime.now().strftime('%Y%m%d-%H%M%S')}",
    description = "This is my test flow for the customer service use case",
    executionRoleArn = flow_role,
    definition = {
      "nodes": [
          {
              "name": "StartNode",
              "type": "Input",
              "configuration": {
                  "input": {}
              },
              "outputs": [
                  {
                      "name": "document",
                      "type": "String"
                  }
              ],
          },
          {
            "name": "Prompt_1",
            "type": "Prompt",
            "configuration": {
              "prompt": {
                "sourceConfiguration": {
                  "resource": {
                      "promptArn": promptArn
                  }
                }
              }
            },
            "inputs": [
              {
                "expression": "$.data",
                "name": "input",
                "type": "String"
              }
            ],
            "outputs": [
              {
                "name": "modelCompletion",
                "type": "String"
              }
            ],
          },
          {
            "name": "EndNode",
            "type": "Output",
            "configuration": {
                "output": {}
            },
            "inputs": [
              {
                "expression": "$.data",
                "name": "document",
                "type": "String"
              }
            ],
          }
      ],
      "connections": [
          {
              "name": "Connection_1",
              "source": "StartNode",
              "target": "Prompt_1",
              "type": "Data",
              "configuration":{
                  "data": {
                      "sourceOutput": "document",
                      "targetInput": "input"
                  }
              }
          },
          {
              "name": "Connection_2",
              "source": "Prompt_1",
              "target": "EndNode",
              "type": "Data",
              "configuration": {
                  "data": {
                      "sourceOutput": "modelCompletion",
                      "targetInput": "document"
                  }
              }
          }
      ],
    }
)
print(json.dumps(response, indent=2, default=str))
flowId = response["id"]
flowArn = response["arn"]
flowName = response["name"]
print(f"Flow ID: {flowId}\nFlow ARN: {flowArn}\nFlow Name: {flowName}")

{
  "ResponseMetadata": {
    "RequestId": "faeddcda-f4d3-4dbd-b085-8397eef0219a",
    "HTTPStatusCode": 201,
    "HTTPHeaders": {
      "date": "Wed, 07 Aug 2024 18:42:07 GMT",
      "content-type": "application/json",
      "content-length": "1330",
      "connection": "keep-alive",
      "x-amzn-requestid": "faeddcda-f4d3-4dbd-b085-8397eef0219a",
      "x-amz-apigw-id": "cJrv8F4cvHcEnFw=",
      "x-amzn-trace-id": "Root=1-66b3bfff-7d03ba4c73ce28026841a2fb"
    },
    "RetryAttempts": 0
  },
  "arn": "arn:aws:bedrock:us-west-2:376567470296:flow/BCDQ95HLQJ",
  "createdAt": "2024-08-07 18:42:07.455653+00:00",
  "definition": {
    "connections": [
      {
        "configuration": {
          "data": {
            "sourceOutput": "document",
            "targetInput": "input"
          }
        },
        "name": "Connection_1",
        "source": "StartNode",
        "target": "Prompt_1",
        "type": "Data"
      },
      {
        "configuration": {
          "data": {
           

Now that we have our first flow, we can prepare it. This basically builds and validates our flow.

In [10]:
response = bedrock_agent.prepare_flow(
    flowIdentifier = flowId
)
print(json.dumps(response, indent=2, default=str))

{
  "ResponseMetadata": {
    "RequestId": "183a99f8-0527-405d-9687-09f6a031adc1",
    "HTTPStatusCode": 202,
    "HTTPHeaders": {
      "date": "Wed, 07 Aug 2024 18:42:12 GMT",
      "content-type": "application/json",
      "content-length": "40",
      "connection": "keep-alive",
      "x-amzn-requestid": "183a99f8-0527-405d-9687-09f6a031adc1",
      "x-amz-apigw-id": "cJrwxEuuvHcESjw=",
      "x-amzn-trace-id": "Root=1-66b3c004-32b299bc349deb583c6b2e85"
    },
    "RetryAttempts": 0
  },
  "id": "BCDQ95HLQJ",
  "status": "Preparing"
}


In [11]:
response = bedrock_agent.get_flow(
    flowIdentifier = flowId
)
print(json.dumps(response, indent=2, default=str))

{
  "ResponseMetadata": {
    "RequestId": "0b08360a-85fb-4205-8d75-35b0e2e4046b",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 07 Aug 2024 18:42:16 GMT",
      "content-type": "application/json",
      "content-length": "1344",
      "connection": "keep-alive",
      "x-amzn-requestid": "0b08360a-85fb-4205-8d75-35b0e2e4046b",
      "x-amz-apigw-id": "cJrxUHPQPHcEqTQ=",
      "x-amzn-trace-id": "Root=1-66b3c008-039fa2d27fe27c3857111046"
    },
    "RetryAttempts": 0
  },
  "arn": "arn:aws:bedrock:us-west-2:376567470296:flow/BCDQ95HLQJ",
  "createdAt": "2024-08-07 18:42:07.455653+00:00",
  "definition": {
    "connections": [
      {
        "configuration": {
          "data": {
            "sourceOutput": "document",
            "targetInput": "input"
          }
        },
        "name": "Connection_1",
        "source": "StartNode",
        "target": "Prompt_1",
        "type": "Data"
      },
      {
        "configuration": {
          "data": {
           

We can also list all the flows in our account.

In [12]:
response = bedrock_agent.list_flows(
    maxResults=10,
)
print(json.dumps(response["flowSummaries"], indent=2, default=str))

[
  {
    "arn": "arn:aws:bedrock:us-west-2:376567470296:flow/ZS4W0YIG0A",
    "createdAt": "2024-07-28 17:15:59.492417+00:00",
    "description": "flow to support customer service ai chatbot",
    "id": "ZS4W0YIG0A",
    "name": "CustomerService",
    "status": "NotPrepared",
    "updatedAt": "2024-08-07 15:38:28.859754+00:00",
    "version": "DRAFT"
  },
  {
    "arn": "arn:aws:bedrock:us-west-2:376567470296:flow/YYCU132JME",
    "createdAt": "2024-08-07 12:31:44.379205+00:00",
    "description": "builds a playlist of songs, given a genre and the number of songs to include in the playlist",
    "id": "YYCU132JME",
    "name": "MakePlaylist",
    "status": "Prepared",
    "updatedAt": "2024-08-07 15:27:06.336108+00:00",
    "version": "DRAFT"
  },
  {
    "arn": "arn:aws:bedrock:us-west-2:376567470296:flow/BCDQ95HLQJ",
    "createdAt": "2024-08-07 18:42:07.455653+00:00",
    "description": "This is my test flow for the customer service use case",
    "id": "BCDQ95HLQJ",
    "name": "M

Let's create a version from our draft flow. Note flow versions are read-only, meaning these cannot be modified once created as they're intended for using in production. If you need to make changes to a flow you can update your draft.

In [13]:
response = bedrock_agent.create_flow_version(
    flowIdentifier = flowId
)
print(json.dumps(response, indent=2, default=str))

{
  "ResponseMetadata": {
    "RequestId": "fe8131de-71c2-443b-9fcb-8ede881c5c1d",
    "HTTPStatusCode": 201,
    "HTTPHeaders": {
      "date": "Wed, 07 Aug 2024 18:42:24 GMT",
      "content-type": "application/json",
      "content-length": "1207",
      "connection": "keep-alive",
      "x-amzn-requestid": "fe8131de-71c2-443b-9fcb-8ede881c5c1d",
      "x-amz-apigw-id": "cJrykGmYPHcEbbg=",
      "x-amzn-trace-id": "Root=1-66b3c010-37c9adcf493e7a8f45d88943"
    },
    "RetryAttempts": 0
  },
  "arn": "arn:aws:bedrock:us-west-2:376567470296:flow/BCDQ95HLQJ",
  "createdAt": "2024-08-07 18:42:24.203549+00:00",
  "definition": {
    "connections": [
      {
        "configuration": {
          "data": {
            "sourceOutput": "document",
            "targetInput": "input"
          }
        },
        "name": "Connection_1",
        "source": "StartNode",
        "target": "Prompt_1",
        "type": "Data"
      },
      {
        "configuration": {
          "data": {
           

We can also create flow alises, so that we can point our application front-ends and any other integrations to these. This allows creating new versions without impacting our service.

In [14]:
response = bedrock_agent.create_flow_alias(
    flowIdentifier = flowId,
    name = flowName,
    description = "Alias for my test flow in the customer service use case",
    routingConfiguration = [
        {
            "flowVersion": "1"
        }
    ]
)
print(json.dumps(response, indent=2, default=str))
flowAliasId = response['id']

{
  "ResponseMetadata": {
    "RequestId": "7f266e40-7ba3-4c15-aac8-2c9df3c2fcc3",
    "HTTPStatusCode": 201,
    "HTTPHeaders": {
      "date": "Wed, 07 Aug 2024 18:42:28 GMT",
      "content-type": "application/json",
      "content-length": "364",
      "connection": "keep-alive",
      "x-amzn-requestid": "7f266e40-7ba3-4c15-aac8-2c9df3c2fcc3",
      "x-amz-apigw-id": "cJrzTEApvHcEKjw=",
      "x-amzn-trace-id": "Root=1-66b3c014-38d5bc7d235cceb251ff4015"
    },
    "RetryAttempts": 0
  },
  "arn": "arn:aws:bedrock:us-west-2:376567470296:flow/BCDQ95HLQJ/alias/1UW3FREB5U",
  "createdAt": "2024-08-07 18:42:28.900560+00:00",
  "description": "Alias for my test flow in the customer service use case",
  "flowId": "BCDQ95HLQJ",
  "id": "1UW3FREB5U",
  "name": "MyTestFlow-20240807-184207",
  "routingConfiguration": [
    {
      "flowVersion": "1"
    }
  ],
  "updatedAt": "2024-08-07 18:42:28.900560+00:00"
}


In [15]:
import json
response = bedrock_agent.list_flow_versions(
    flowIdentifier = flowId,
    maxResults = 10
)
print(json.dumps(response, indent=2, default=str))

{
  "ResponseMetadata": {
    "RequestId": "b229f8bf-263e-4143-9cb0-27936a81c970",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 07 Aug 2024 18:42:34 GMT",
      "content-type": "application/json",
      "content-length": "354",
      "connection": "keep-alive",
      "x-amzn-requestid": "b229f8bf-263e-4143-9cb0-27936a81c970",
      "x-amz-apigw-id": "cJr0NE2-PHcEp2Q=",
      "x-amzn-trace-id": "Root=1-66b3c01a-48fe20d515200a4d790e5ce8"
    },
    "RetryAttempts": 0
  },
  "flowVersionSummaries": [
    {
      "arn": "arn:aws:bedrock:us-west-2:376567470296:flow/BCDQ95HLQJ",
      "createdAt": "2024-08-07 18:42:07.455653+00:00",
      "id": "BCDQ95HLQJ",
      "status": "Prepared",
      "version": "DRAFT"
    },
    {
      "arn": "arn:aws:bedrock:us-west-2:376567470296:flow/BCDQ95HLQJ",
      "createdAt": "2024-08-07 18:42:24.203549+00:00",
      "id": "BCDQ95HLQJ",
      "status": "Prepared",
      "version": "1"
    }
  ]
}


In [16]:
response = bedrock_agent.get_flow_version(
    flowIdentifier = flowId,
    flowVersion = '1'
)
print(json.dumps(response, indent=2, default=str))

{
  "ResponseMetadata": {
    "RequestId": "adbbcc6c-0fda-4196-ac60-8f6f8a345edc",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 07 Aug 2024 18:42:38 GMT",
      "content-type": "application/json",
      "content-length": "1207",
      "connection": "keep-alive",
      "x-amzn-requestid": "adbbcc6c-0fda-4196-ac60-8f6f8a345edc",
      "x-amz-apigw-id": "cJr0uGEKvHcEANA=",
      "x-amzn-trace-id": "Root=1-66b3c01d-514770df44edb298514a4e6e"
    },
    "RetryAttempts": 0
  },
  "arn": "arn:aws:bedrock:us-west-2:376567470296:flow/BCDQ95HLQJ",
  "createdAt": "2024-08-07 18:42:24.203549+00:00",
  "definition": {
    "connections": [
      {
        "configuration": {
          "data": {
            "sourceOutput": "document",
            "targetInput": "input"
          }
        },
        "name": "Connection_1",
        "source": "StartNode",
        "target": "Prompt_1",
        "type": "Data"
      },
      {
        "configuration": {
          "data": {
           

We can also update a given alias assigned to a flow, for e.g. pointing to another version if required.

In [17]:
response = bedrock_agent.update_flow_alias(
    flowIdentifier = flowId,
    aliasIdentifier = flowAliasId,
    name = flowName,
    routingConfiguration = [
        {
            "flowVersion": "1"
        }
    ]
)
flowAliasId = response["id"]
print(json.dumps(response, indent=2, default=str))

{
  "ResponseMetadata": {
    "RequestId": "5745e3ac-7a00-4765-a01c-aa25fc27145e",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 07 Aug 2024 18:42:43 GMT",
      "content-type": "application/json",
      "content-length": "247",
      "connection": "keep-alive",
      "x-amzn-requestid": "5745e3ac-7a00-4765-a01c-aa25fc27145e",
      "x-amz-apigw-id": "cJr1mEq7PHcEgiA=",
      "x-amzn-trace-id": "Root=1-66b3c023-36fbeb9846c79f6d42a202df"
    },
    "RetryAttempts": 0
  },
  "arn": "arn:aws:bedrock:us-west-2:376567470296:flow/BCDQ95HLQJ/alias/1UW3FREB5U",
  "flowId": "BCDQ95HLQJ",
  "id": "1UW3FREB5U",
  "name": "MyTestFlow-20240807-184207",
  "routingConfiguration": [
    {
      "flowVersion": "1"
    }
  ],
  "updatedAt": "2024-08-07 18:42:28.900560+00:00"
}


In [18]:
response = bedrock_agent.get_flow_alias(
    flowIdentifier = flowId,
    aliasIdentifier = flowAliasId
)
print(json.dumps(response, indent=2, default=str))

{
  "ResponseMetadata": {
    "RequestId": "ab7428fa-0865-4819-950c-5c76c8a4ca18",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 07 Aug 2024 18:42:47 GMT",
      "content-type": "application/json",
      "content-length": "292",
      "connection": "keep-alive",
      "x-amzn-requestid": "ab7428fa-0865-4819-950c-5c76c8a4ca18",
      "x-amz-apigw-id": "cJr2KGkavHcEH0A=",
      "x-amzn-trace-id": "Root=1-66b3c027-6e394aae1e530e9c4f44e982"
    },
    "RetryAttempts": 0
  },
  "arn": "arn:aws:bedrock:us-west-2:376567470296:flow/BCDQ95HLQJ/alias/1UW3FREB5U",
  "createdAt": "2024-08-07 18:42:28.900560+00:00",
  "flowId": "BCDQ95HLQJ",
  "id": "1UW3FREB5U",
  "name": "MyTestFlow-20240807-184207",
  "routingConfiguration": [
    {
      "flowVersion": "1"
    }
  ],
  "updatedAt": "2024-08-07 18:42:28.900560+00:00"
}


### Invoke a Flow

Now that we have learned how to create and manage flows, we can test these with invocations.

Note for this we'll rely on the Bedrock Agent Runtime SDK.

You can invoke flows from any application front-end or your own systems as required. It effectively exposes all the logic of your flow through an Agent Endpoint API.

In [20]:
bedrock_agent_runtime = boto3.client(service_name = 'bedrock-agent-runtime', region_name = 'us-west-2')

In [21]:
response = bedrock_agent_runtime.invoke_flow(
    flowIdentifier = flowId,
    flowAliasIdentifier = flowAliasId,
    inputs = [
        { 
            "content": { 
                "document": "Hi, I need help with my order!"
            },
            "nodeName": "StartNode",
            "nodeOutputName": "document"
        }
    ]
)
event_stream = response["responseStream"]
for event in event_stream:
    print(json.dumps(event, indent=2, ensure_ascii=False))

EventStreamError: An error occurred (accessDeniedException) when calling the InvokeFlow operation: Access denied. Check your request permissions and retry the request.

---

### Cleaning-up Resources (optional)

Before leaving, here's how to delete the resources that we've created.

In [ ]:
response = bedrock_agent.delete_flow_alias(
    flowIdentifier = flowId,
    aliasIdentifier = flowAliasId
)
print(json.dumps(response, indent=2, default=str))

In [ ]:
response = bedrock_agent.delete_flow_version(
    flowIdentifier = flowId,
    flowVersion = '1'
)
print(json.dumps(response, indent=2, default=str))

In [ ]:
response = bedrock_agent.delete_flow(
    flowIdentifier = flowId
)
print(json.dumps(response, indent=2, default=str))

In [ ]:
response = bedrock_agent.delete_prompt(
    promptIdentifier = promptId
)
print(json.dumps(response, indent=2, default=str))

-------